In [1]:
#@title Install dependencies. { display-mode: "form" }
!pip3 install torch torchvision torchaudio transformers sentencepiece accelerate --extra-index-url https://download.pytorch.org/whl/cu116
!python -m pip install ankh

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.3 MB/s eta 0:00:00


In [2]:
#@title Import dependencies. { display-mode: "form" }
from transformers import BertForMaskedLM, BertTokenizer, pipeline
from transformers import AutoModel, AutoTokenizer
from sklearn.manifold import TSNE
from google.colab import drive
import torch, os, pandas as pd, random, matplotlib.pyplot as plt, numpy as np
import seaborn as sns
import time, random, re, shutil
import ankh

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("Using device: {}".format(device))

drive.mount('/content/drive/')
path = '/content/drive/MyDrive/msc_project/all_paired'
os.chdir(path)
os.listdir()

Using device: cuda:0
Mounted at /content/drive/


['230604_human_paired_seqs.csv',
 '230604_random_100.csv',
 'opig_data',
 '230618_human_paired_seqs.csv',
 'paired_human.fasta',
 '230624_test.csv',
 '230625_human_paired_seqs.csv',
 '230625_paired_rand1000.csv',
 '230701_dim_reduced_TSNE_rand1000.parquet',
 '230701_protein_embeddings.parquet',
 '230716_heavy_chain_protein_embeddings_1000.parquet',
 '230716_human_paired_seqs.csv',
 '230716_scfv_10000.csv',
 '230716_scfv_10000.fasta',
 'new_tmp',
 'clusterRes_all_seqs.fasta',
 'clusterRes_rep_seq.fasta',
 'clusterRes_cluster.tsv',
 '230722_cluster_tensor_scfv_10000.csv']

In [3]:
df = pd.read_csv('230716_human_paired_seqs.csv', dtype={'Run':np.str_}, index_col=0)
df.columns

Index(['Unnamed: 1', 'sequence_id_heavy', 'ANARCI_status_heavy',
       'sequence_heavy', 'sequence_alignment_aa_heavy', 'v_call_heavy',
       'd_call_heavy', 'j_call_heavy', 'sequence_id_light',
       'ANARCI_status_light', 'sequence_light', 'v_call_light', 'j_call_light',
       'sequence_alignment_aa_light', 'Run', 'seq_id', 'scfv'],
      dtype='object')

In [4]:
protein_sequences = [df['scfv'].iloc[1], df['scfv'].iloc[2]]

In [12]:
df.head()

,Unnamed: 1,sequence_id_heavy,ANARCI_status_heavy,sequence_heavy,sequence_alignment_aa_heavy,v_call_heavy,d_call_heavy,j_call_heavy,sequence_id_light,ANARCI_status_light,sequence_light,v_call_light,j_call_light,sequence_alignment_aa_light,Run,seq_id,scfv
0,0,AAACCTGAGACTAAGT-1_contig_3,"|Deletions: 10, 73||||",AGCTCTCAGAGAGGTGCCTTAGCCCTGGATTCCAAGGCATTTCCAC...,EVQLVESGGGLVQPGGSLRLSCAASGFTFSSYSMNWVRQAPGKGLE...,IGHV3-48*02,IGHD3-3*01,IGHJ6*02,AAACCTGAGACTAAGT-1_contig_1,|||||,GCTGTGCTGTGGGTCCAGGAGGCAGAACTCTGGGTGTCTCACCATG...,IGLV3-25*03,IGLJ2*01,SYELTQPPSVSVSPGQTARITCSGDALPKQYAYWYQQKPGQAPVLV...,1287185,AAACCTGAGACTAAGT-1_contig_3_AAACCTGAGACTAAGT-1...,SYELTQPPSVSVSPGQTARITCSGDALPKQYAYWYQQKPGQAPVLV...
1,1,AAACCTGAGAGGGCTT-1_contig_1,"|Deletions: 10, 73||||",TGGGGACCCAAAAACCACACCCCTCCTTGGGAGAATCCCCTAGATC...,QVQLVQSGAEVKKPGASVKVSCKASGYTFTSYGISWVRQAPGQGLE...,IGHV1-18*01,IGHD2-2*01,IGHJ6*02,AAACCTGAGAGGGCTT-1_contig_2,|||||,AGAGCTCTGGAGAAGAGCTGCTCAGTTAGGACCCAGAGGGAACCAT...,IGKV3-20*01,IGKJ1*01,EIVLTQSPGTLSLSPGERATLSCRASQSVSSSYLAWYQQKPGQAPR...,1287185,AAACCTGAGAGGGCTT-1_contig_1_AAACCTGAGAGGGCTT-1...,EIVLTQSPGTLSLSPGERATLSCRASQSVSSSYLAWYQQKPGQAPR...
2,2,AAACCTGAGAGTCTGG-1_contig_1,"|Deletions: 10, 73||||",GCAGAGCCTGCTGAATTCTGGCTGACCAGGGCAGTCACCAGAGCTC...,QVQLQQSGPGLVKPSQTLSLTCAISGDSVSSNSAAWNWIRQSPSRG...,IGHV6-1*01,IGHD1-1*01,IGHJ6*02,AAACCTGAGAGTCTGG-1_contig_2,|||||,CTGGGCCTCAGGAAGCAGCATCGGAGGTGCCTCAGCCATGGCATGG...,IGLV3-1*01,IGLJ2*01,SYELTQPPSVSVSPGQTASITCSGDKLGDKYACWYQQKPGQSPVLV...,1287185,AAACCTGAGAGTCTGG-1_contig_1_AAACCTGAGAGTCTGG-1...,SYELTQPPSVSVSPGQTASITCSGDKLGDKYACWYQQKPGQSPVLV...
3,3,AAACCTGAGCGTAATA-1_contig_2,"|Deletions: 10, 73||||",AGCTCTGAGAGAGGAGCCCAGCCCTGGGATTTTCAGGTGTTTTCAT...,EVQLLESGGGLVQPGGSLRLSCAASGFTFSSYAMSWVRQAPGKGLE...,IGHV3-23*01,IGHD2-15*01,IGHJ4*02,AAACCTGAGCGTAATA-1_contig_1,|||||,CTGGGCCTCAGGAAGCAGCATCGGAGGTGCCTCAGCCATGGCATGG...,IGLV3-1*01,IGLJ2*01,SYELTQPPSVSVSPGQTASITCSGDKLGDKYACWYQQKPGQSPVLV...,1287185,AAACCTGAGCGTAATA-1_contig_2_AAACCTGAGCGTAATA-1...,SYELTQPPSVSVSPGQTASITCSGDKLGDKYACWYQQKPGQSPVLV...
4,4,AAACCTGAGCTAACAA-1_contig_2,"|Deletions: 10, 73||||",GAGAGAGGAGCCCAGCCCTGGGATTTTCAGGTGTTTTCATTTGGTG...,EVQLLESGGGLVQPGGSLRLSCAASGFTFSSYAMSWVRQAPGKGLE...,IGHV3-23*01,IGHD4-17*01,IGHJ4*02,AAACCTGAGCTAACAA-1_contig_1,|||||,AGAGCTCTGGAGAAGAGCTGCTCAGTTAGGACCCAGAGGGAACCAT...,IGKV3-20*01,IGKJ1*01,EIVLTQSPGTLSLSPGERATLSCRASQSVSSSYLAWYQQKPGQAPR...,1287185,AAACCTGAGCTAACAA-1_contig_2_AAACCTGAGCTAACAA-1...,EIVLTQSPGTLSLSPGERATLSCRASQSVSSSYLAWYQQKPGQAPR...


In [5]:
model, tokenizer = ankh.load_large_model()
model.eval()

model.to(device=device)

T5EncoderModel(
  (shared): Embedding(144, 1536)
  (encoder): T5Stack(
    (embed_tokens): Embedding(144, 1536)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1536, out_features=1024, bias=False)
              (k): Linear(in_features=1536, out_features=1024, bias=False)
              (v): Linear(in_features=1536, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1536, bias=False)
              (relative_attention_bias): Embedding(64, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1536, out_features=3840, bias=False)
              (wi_1): Linear(in_features=1536, out_features=3840, bias=False)
              (wo): Lin

In [6]:
# protein_sequences = [list(seq) for seq in protein_sequences]
seqs = [" ".join(list(re.sub(r"[UZOB]", "X", sequence))) for sequence in protein_sequences]
# tokenize sequences and pad up to the longest sequence in the batch
ids = tokenizer.batch_encode_plus(seqs, add_special_tokens=True, padding="longest")
input_ids = torch.tensor(ids['input_ids']).to(device)
attention_mask = torch.tensor(ids['attention_mask']).to(device)
# generate embeddings
with torch.no_grad():
    embedding_repr = model(input_ids=input_ids,attention_mask=attention_mask)
ember = embedding_repr.last_hidden_state
#derivation of per-protein embedding for all embedded proteins
ember = [i.mean(dim=0) for i in ember]

In [6]:
def process_seqs(seqs: list) -> list:
  # this will replace all rare/ambiguous amino acids by X and introduce white-space between all amino acids
  seqs = [" ".join(list(re.sub(r"[UZOB]", "X", sequence))) for sequence in seqs]
  # tokenize sequences and pad up to the longest sequence in the batch
  ids = tokenizer.batch_encode_plus(seqs, add_special_tokens=True, padding="longest")
  input_ids = torch.tensor(ids['input_ids']).to(device)
  attention_mask = torch.tensor(ids['attention_mask']).to(device)
  # generate embeddings
  with torch.no_grad():
      embedding_repr = model(input_ids=input_ids,attention_mask=attention_mask)
  ember = embedding_repr.last_hidden_state
  #derivation of per-protein embedding for all embedded proteins
  ember = [i.mean(dim=0) for i in ember]
  return ember


In [7]:
prott5 = pd.read_csv('230716_scfv_10000.csv', index_col=0)
names = prott5['seq']

df = df[df['seq_id'].isin(names)]

df.head()

,Unnamed: 1,sequence_id_heavy,ANARCI_status_heavy,sequence_heavy,sequence_alignment_aa_heavy,v_call_heavy,d_call_heavy,j_call_heavy,sequence_id_light,ANARCI_status_light,sequence_light,v_call_light,j_call_light,sequence_alignment_aa_light,Run,seq_id,scfv
360,360,AACGTTGTCTCGCATC-1_contig_2,"|Deletions: 10, 73||||",AGCTCTGGGAGAAGAGCCCCAGCCCCAGAATTCCCAGGAGTTTCCA...,QVQLVESGGGLVKPGGSLRLSCAASGFTFSDYYMSWIRQAPGKGLE...,IGHV3-11*01,IGHD3-22*01,IGHJ5*02,AACGTTGTCTCGCATC-1_contig_1,|||||,GGGGAGGAACTGCTCAGTTAGGACCCAGACGGAACCATGGAAGCCC...,IGKV3-15*01,IGKJ1*01,EIVMTQSPATLSVSPGERATLSCRASQSVSSNLAWYQQKPGQAPRL...,1287185,AACGTTGTCTCGCATC-1_contig_2_AACGTTGTCTCGCATC-1...,EIVMTQSPATLSVSPGERATLSCRASQSVSSNLAWYQQKPGQAPRL...
413,413,AACTCCCCAGTTTACG-1_contig_1,"|Deletions: 10, 73||||",AGCTCTGGGAGAGGAGCCCCAGCCCTGAGATTCCCAGGTGTTTCCA...,EVQLVESGGGVVQPGGSLRLSCAASGFTFDDYAMHWVRQAPGKGLE...,IGHV3-43*02,IGHD1-26*01,IGHJ6*02,AACTCCCCAGTTTACG-1_contig_2,|||||,GGGAAGCTCAGCTTCAGCTGTGGTAGAGAAGACAGGATTCAGGACA...,IGLV1-47*01,IGLJ2*01,QSVLTQPPSASGTPGQRVTISCSGSSSNIGSNYVYWYQQLPGTAPK...,1287185,AACTCCCCAGTTTACG-1_contig_1_AACTCCCCAGTTTACG-1...,QSVLTQPPSASGTPGQRVTISCSGSSSNIGSNYVYWYQQLPGTAPK...
513,514,AACTTTCCAATCTACG-1_contig_2,"|Deletions: 10, 73||||",GGGGAGCTCTGGGAGAGGAGCCCCAGCCCTGAGATTCCCAGGTGTT...,EVQLVESGGGLVQPGRSLRLSCAASGFTFDDYAMHWVRQAPGKGLE...,IGHV3-9*01,IGHD2-15*01,IGHJ3*02,AACTTTCCAATCTACG-1_contig_1,|||||,TGAGCGCAGAAGGCAGGACTCGGGACAATCTTCATCATGACCTGCT...,IGLV1-51*01,IGLJ2*01,QSVLTQPPSVSAAPGQKVTISCSGSSSNIGNNYVSWYQQLPGTAPK...,1287185,AACTTTCCAATCTACG-1_contig_2_AACTTTCCAATCTACG-1...,QSVLTQPPSVSAAPGQKVTISCSGSSSNIGNNYVSWYQQLPGTAPK...
683,684,AAGGTTCGTCAGAGGT-1_contig_2,"|Deletions: 10, 73||||",CTCTTGTCTTATCTGGGAGCTCTGGGAGAGGAGCCCCAGCCCTGAG...,EVQLVESGGGLVQPGRSLRLSCAASGFTFDDYAMHWVRQAPGKGLE...,IGHV3-9*01,IGHD6-19*01,IGHJ1*01,AAGGTTCGTCAGAGGT-1_contig_1,|||||,CTCTGGGGAGAGCTCTGGAGAAGAGCTGCTCAGTTAGGACCCAGAG...,IGKV3-20*01,IGKJ2*01,EIVLTQSPGTLSLSPGERATLSCRASQSVSSSYLAWYQQKPGQAPR...,1287185,AAGGTTCGTCAGAGGT-1_contig_2_AAGGTTCGTCAGAGGT-1...,EIVLTQSPGTLSLSPGERATLSCRASQSVSSSYLAWYQQKPGQAPR...
828,830,ACACCAACACAACTGT-1_contig_2,"|Deletions: 10, 73||||",AGCTCTGGGAGAAGAGCCCCAGCCCCAGAATTCCCAGGAGTTTCCA...,QVQLVESGGGLVKPGGSLRLSCAASGFTFSDYYMSWIRQAPGKGLE...,IGHV3-11*01,IGHD1-1*01,IGHJ5*02,ACACCAACACAACTGT-1_contig_1,|||||,AGCCTGGGCCTCAGGAAGCAGCATCGGGGGTGCCGCAGCCATGGCC...,IGLV3-9*01,IGLJ2*01,SYELTQPLSVSVALGQTARITCGGNNIGSKNVHWYQQKPGQAPVLV...,1287185,ACACCAACACAACTGT-1_contig_2_ACACCAACACAACTGT-1...,SYELTQPLSVSVALGQTARITCGGNNIGSKNVHWYQQKPGQAPVLV...


In [8]:
l = []
batch_size = 100
for i in range(0, df.shape[0], batch_size):
    df1 = df.iloc[i:i + batch_size]
    batch = df1['scfv'].to_list()
    batch = process_seqs(batch)
    # Convert protein_embeddings to a numpy array
    protein_embeddings_np = np.array([emb.cpu().numpy() for emb in batch])
    # Save protein_embeddings to a Parquet file
    df_tensor = pd.DataFrame(protein_embeddings_np)
    df_tensor['seq'] = df1['seq_id'].to_list()
    l.append(df_tensor)

In [9]:
len(l)

100

In [10]:
tensor_df = pd.concat(l)
tensor_df.head()

,0,1,2,3,4,5,6,7,8,9,...,1527,1528,1529,1530,1531,1532,1533,1534,1535,seq
0,-0.024482,0.007228,-0.001585,0.000707,0.008215,0.004985,0.003505,0.001370,-0.006802,0.001784,...,-0.012195,0.002781,-0.005075,-0.001047,0.009039,0.000002,-0.009873,-0.002028,-0.007518,AACGTTGTCTCGCATC-1_contig_2_AACGTTGTCTCGCATC-1...
1,-0.025085,0.008314,-0.001044,0.001571,0.006498,0.006720,0.003649,0.002229,-0.007137,0.002653,...,-0.011894,0.002423,-0.006037,-0.002191,0.009906,-0.001431,-0.008674,-0.001290,-0.006119,AACTCCCCAGTTTACG-1_contig_1_AACTCCCCAGTTTACG-1...
2,-0.024743,0.004909,-0.003940,0.002075,0.012859,0.005442,0.005055,0.004185,-0.007867,0.003407,...,-0.010496,0.000440,-0.006551,-0.003698,0.005067,-0.003032,-0.009678,-0.001725,-0.005766,AACTTTCCAATCTACG-1_contig_2_AACTTTCCAATCTACG-1...
3,-0.022117,0.007103,-0.002866,0.001766,0.009317,0.004625,0.003328,0.000341,-0.005280,0.001050,...,-0.011916,0.001629,-0.006447,-0.003057,0.008038,-0.003565,-0.005637,-0.000655,-0.007553,AAGGTTCGTCAGAGGT-1_contig_2_AAGGTTCGTCAGAGGT-1...
4,-0.024954,0.009146,-0.003355,-0.001528,0.001854,0.007528,0.002048,0.001132,-0.011293,0.001699,...,-0.012107,0.004293,-0.006576,-0.001942,0.007095,0.000418,-0.008124,-0.002870,-0.005733,ACACCAACACAACTGT-1_contig_2_ACACCAACACAACTGT-1...


In [11]:
tensor_df.shape

(9955, 1537)

In [12]:
tensor_df.to_csv('230805_ankh_tensor_10000.csv', index=False)